In [358]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn import tree
df = pd.read_csv('city_temperature.csv')
df.head()
print(df.columns)


Index(['Region', 'Country', 'State', 'City', 'Month', 'Day', 'Year',
       'AvgTemperature'],
      dtype='object')


In [359]:
df=df[df['City']=='Frankfurt']

 

In [360]:
df = df.replace(-99.0, None)
df = df.bfill()
x=df['AvgTemperature'].isna().sum()
print(x)

0


In [361]:
df = df[['Month','Day','Year','AvgTemperature']]
df['AvgTemperature'] = (df['AvgTemperature'] - 32) / 1.8  # celcius!!!!
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df[ ['Date','AvgTemperature'] ]
df=df.bfill()

In [362]:
def linreg(X, Y):
    """
    return a,b in solution to y = ax + b such that root mean square distance between trend line and original points is minimized
    """
    N = len(X)
    Sx = Sy = Sxx = Syy = Sxy = 0.0
    for x, y in zip(X, Y):
        Sx = Sx + x
        Sy = Sy + y
        Sxx = Sxx + x*x
        Syy = Syy + y*y
        Sxy = Sxy + x*y
    det = Sxx * N - Sx * Sx
    return (Sxy * N - Sy * Sx)/det, (Sxx * Sy - Sx * Sxy)/det




In [363]:
x = df.dropna()['AvgTemperature'].values
m,n= linreg(range(len(x)),x) 
df['x'] = range(len(df))# create an index
df['TREND']=df['x']*( -0.000367753777687857) +12.209758723893017
df['DE-TREND'] = df['AvgTemperature'] - df['TREND']
df['DE-TREND'] 

734085   -11.598648
734086   -11.598280
734087   -11.597912
734088   -16.541989
734089   -18.097177
            ...    
738216     2.087210
738217     3.920911
738218     3.532390
738219     4.088313
738220     2.199792
Name: DE-TREND, Length: 4136, dtype: float64

In [364]:
for i in  [1,365]:
    print(i,df["DE-TREND"].autocorr(i))


1 0.9516102934683462
365 0.6759997185298405


In [365]:
for i in [1,365]:
    df[f'lag{i}']=df['DE-TREND'].shift(i)
df

,Date,AvgTemperature,x,TREND,DE-TREND,lag1,lag365
734085,1995-01-01,0.611111,0,12.209759,-11.598648,NaN,NaN
734086,1995-01-02,0.611111,1,12.209391,-11.598280,-11.598648,NaN
734087,1995-01-03,0.611111,2,12.209023,-11.597912,-11.598280,NaN
734088,1995-01-04,-4.333333,3,12.208655,-16.541989,-11.597912,NaN
734089,1995-01-05,-5.888889,4,12.208288,-18.097177,-16.541989,NaN
...,...,...,...,...,...,...,...
738216,2006-04-24,12.777778,4131,10.690568,2.087210,2.086842,1.341869
738217,2006-04-25,14.611111,4132,10.690200,3.920911,2.087210,1.342236
738218,2006-04-26,14.222222,4133,10.689832,3.532390,3.920911,1.620382
738219,2006-04-27,14.777778,4134,10.689465,4.088313,3.532390,3.120750


In [366]:
# for r in [3,4,5,7,14,30,31]:
#     for r2 in [3,4,5,7,14,30,31]:
#         df[f'MA{r}-{r2}']=(df["DE-TREND"].rolling(r+r2).sum()-df['DE-TREND'].rolling(r2).sum())/(r)

df['MAP7-3'] = (df['DE-TREND'].rolling(10).sum() - df['DE-TREND'].rolling(3).sum()) / (7)
df = df.dropna()
df


,Date,AvgTemperature,x,TREND,DE-TREND,lag1,lag365,MAP7-3
734450,1996-01-01,-0.611111,365,12.075529,-12.686640,-12.687007,-11.598648,-11.926941
734451,1996-01-02,-0.611111,366,12.075161,-12.686272,-12.686640,-11.598280,-12.323399
734452,1996-01-03,14.000000,367,12.074793,1.925207,-12.686272,-11.597912,-12.719857
734453,1996-01-04,14.000000,368,12.074425,1.925575,1.925207,-16.541989,-13.124251
734454,1996-01-05,14.000000,369,12.074058,1.925942,1.925575,-18.097177,-13.401661
...,...,...,...,...,...,...,...,...
738216,2006-04-24,12.777778,4131,10.690568,2.087210,2.086842,1.341869,1.481829
738217,2006-04-25,14.611111,4132,10.690200,3.920911,2.087210,1.342236,1.363149
738218,2006-04-26,14.222222,4133,10.689832,3.532390,3.920911,1.620382,1.244469
738219,2006-04-27,14.777778,4134,10.689465,4.088313,3.532390,3.120750,1.125789


In [367]:
df['Month']=df['Date'].dt.month
month_dummies = pd.get_dummies(df['Month'], prefix='Month', drop_first=True)

# Concatenate the original DataFrame and the dummy variables
df = pd.concat([df, month_dummies], axis=1)
correlation_matrix =df.corr()
correlation_matrix

C:\Users\User\AppData\Local\Temp\ipykernel_17568\2711848158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month']=df['Date'].dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_17568\2711848158.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix =df.corr()


,AvgTemperature,x,TREND,DE-TREND,lag1,lag365,MAP7-3,Month,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
AvgTemperature,1.000000,-0.071336,0.071336,0.998529,0.951607,0.674356,0.852056,0.136988,-0.264237,-0.134663,0.006028,0.164391,0.278084,0.313611,0.349515,0.182627,0.019734,-0.185094,-0.353638
x,-0.071336,1.000000,-1.000000,-0.017149,-0.016481,0.000379,-0.014026,0.032155,-0.004751,0.004608,0.009977,-0.029060,-0.020309,-0.012289,-0.003766,0.004537,0.013004,0.021011,0.029774
TREND,0.071336,-1.000000,1.000000,0.017149,0.016481,-0.000379,0.014026,-0.032155,0.004751,-0.004608,-0.009977,0.029060,0.020309,0.012289,0.003766,-0.004537,-0.013004,-0.021011,-0.029774
DE-TREND,0.998529,-0.017149,0.017149,1.000000,0.953001,0.676000,0.853344,0.139065,-0.265132,-0.134736,0.006585,0.163207,0.277650,0.313697,0.350151,0.183313,0.020488,-0.184397,-0.352871
lag1,0.951607,-0.016481,0.016481,0.953001,1.000000,0.676453,0.874767,0.149705,-0.268278,-0.142926,0.000672,0.161688,0.276474,0.310176,0.353773,0.188056,0.026617,-0.178308,-0.350318
lag365,0.674356,0.000379,-0.000379,0.676000,0.676453,1.000000,0.691589,0.120252,-0.251029,-0.135513,0.007467,0.167706,0.271624,0.320398,0.349495,0.174000,0.025101,-0.195660,-0.365065
MAP7-3,0.852056,-0.014026,0.014026,0.853344,0.874767,0.691589,1.000000,0.217240,-0.296402,-0.187094,-0.029310,0.159252,0.273557,0.310912,0.384448,0.224159,0.054629,-0.146725,-0.348169
Month,0.136988,0.032155,-0.032155,0.139065,0.149705,0.120252,0.217240,1.000000,-0.379510,-0.308327,-0.212936,-0.120286,-0.033497,0.052087,0.138273,0.220491,0.310645,0.389816,0.483018
Month_2,-0.264237,-0.004751,0.004751,-0.265132,-0.268278,-0.251029,-0.296402,-0.379510,1.000000,-0.094531,-0.092536,-0.089727,-0.088140,-0.089727,-0.089727,-0.088140,-0.089727,-0.088140,-0.089727
Month_3,-0.134663,0.004608,-0.004608,-0.134736,-0.142926,-0.135513,-0.187094,-0.308327,-0.094531,1.000000,-0.097319,-0.094365,-0.092697,-0.094365,-0.094365,-0.092697,-0.094365,-0.092697,-0.094365


In [368]:
df["MONTH1-6,7,8"]=df['Month'].isin([6,7,8,9,10])

correlation_matrix =df.corr()
correlation_matrix

C:\Users\User\AppData\Local\Temp\ipykernel_17568\1562651751.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix =df.corr()


,AvgTemperature,x,TREND,DE-TREND,lag1,lag365,MAP7-3,Month,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,"MONTH1-6,7,8"
AvgTemperature,1.000000,-0.071336,0.071336,0.998529,0.951607,0.674356,0.852056,0.136988,-0.264237,-0.134663,0.006028,0.164391,0.278084,0.313611,0.349515,0.182627,0.019734,-0.185094,-0.353638,0.635878
x,-0.071336,1.000000,-1.000000,-0.017149,-0.016481,0.000379,-0.014026,0.032155,-0.004751,0.004608,0.009977,-0.029060,-0.020309,-0.012289,-0.003766,0.004537,0.013004,0.021011,0.029774,-0.010399
TREND,0.071336,-1.000000,1.000000,0.017149,0.016481,-0.000379,0.014026,-0.032155,0.004751,-0.004608,-0.009977,0.029060,0.020309,0.012289,0.003766,-0.004537,-0.013004,-0.021011,-0.029774,0.010399
DE-TREND,0.998529,-0.017149,0.017149,1.000000,0.953001,0.676000,0.853344,0.139065,-0.265132,-0.134736,0.006585,0.163207,0.277650,0.313697,0.350151,0.183313,0.020488,-0.184397,-0.352871,0.636843
lag1,0.951607,-0.016481,0.016481,0.953001,1.000000,0.676453,0.874767,0.149705,-0.268278,-0.142926,0.000672,0.161688,0.276474,0.310176,0.353773,0.188056,0.026617,-0.178308,-0.350318,0.642293
lag365,0.674356,0.000379,-0.000379,0.676000,0.676453,1.000000,0.691589,0.120252,-0.251029,-0.135513,0.007467,0.167706,0.271624,0.320398,0.349495,0.174000,0.025101,-0.195660,-0.365065,0.634352
MAP7-3,0.852056,-0.014026,0.014026,0.853344,0.874767,0.691589,1.000000,0.217240,-0.296402,-0.187094,-0.029310,0.159252,0.273557,0.310912,0.384448,0.224159,0.054629,-0.146725,-0.348169,0.693822
Month,0.136988,0.032155,-0.032155,0.139065,0.149705,0.120252,0.217240,1.000000,-0.379510,-0.308327,-0.212936,-0.120286,-0.033497,0.052087,0.138273,0.220491,0.310645,0.389816,0.483018,0.383307
Month_2,-0.264237,-0.004751,0.004751,-0.265132,-0.268278,-0.251029,-0.296402,-0.379510,1.000000,-0.094531,-0.092536,-0.089727,-0.088140,-0.089727,-0.089727,-0.088140,-0.089727,-0.088140,-0.089727,-0.247723
Month_3,-0.134663,0.004608,-0.004608,-0.134736,-0.142926,-0.135513,-0.187094,-0.308327,-0.094531,1.000000,-0.097319,-0.094365,-0.092697,-0.094365,-0.094365,-0.092697,-0.094365,-0.092697,-0.094365,-0.260528


In [369]:
TREND=df['TREND']
del df['TREND']
del df['x']
del df['Date']
del df['AvgTemperature']

columns_to_keep = [col for col in df.columns if col == 'MONTH1-6,7,8' or not col.startswith('Month')]

# Create a new DataFrame with only the desired columns
df = df[columns_to_keep]

# Print the modified DataFrame
df

,DE-TREND,lag1,lag365,MAP7-3,"MONTH1-6,7,8"
734450,-12.686640,-12.687007,-11.598648,-11.926941,False
734451,-12.686272,-12.686640,-11.598280,-12.323399,False
734452,1.925207,-12.686272,-11.597912,-12.719857,False
734453,1.925575,1.925207,-16.541989,-13.124251,False
734454,1.925942,1.925575,-18.097177,-13.401661,False
...,...,...,...,...,...
738216,2.087210,2.086842,1.341869,1.481829,False
738217,3.920911,2.087210,1.342236,1.363149,False
738218,3.532390,3.920911,1.620382,1.244469,False
738219,4.088313,3.532390,3.120750,1.125789,False


In [370]:
from sklearn.ensemble import RandomForestRegressor
y = df['DE-TREND']
df['MONTH1-6,7,8']=df['MONTH1-6,7,8'].astype(float)
X = df.drop(columns=['DE-TREND'])

print(X.dtypes)
print(y.dtypes)

regr = RandomForestRegressor(max_depth=5)
regr.fit(X, y)
print( regr.score(X, y) )

lag1            float64
lag365          float64
MAP7-3          float64
MONTH1-6,7,8    float64
dtype: object
float64
0.9199660661114044
